In [ ]:
import numpy as np
import pandas as pd


def permutation_test_max(id2, id1, n_permutations=100000, df=full):
    X = df[df['Batch'] == id2][['y']].to_numpy()
    Y = df[df['Batch'] == id1][['y']].to_numpy()

    np.random.seed(0)
    combined = np.concatenate([X, Y])
    n = len(X)
    obs_diff = np.max(X) - np.max(Y)
    
    perm_diffs = []
    for _ in range(n_permutations):
        np.random.shuffle(combined)
        X_permuted = combined[:n]
        Y_permuted = combined[n:]
        max_diff = np.max(X_permuted) - np.max(Y_permuted)
        perm_diffs.append(max_diff)
        
    p_value = np.mean(perm_diffs > obs_diff)
    return np.array(perm_diffs), obs_diff, p_value

In [6]:
import numpy as np
import pandas as pd
from bayesian_optimization import BayesianOptimization
from botorch.test_functions import Michalewicz

def permutation_test_max(id2, id1, n_permutations=10000, df=None):
    X = df[df['Batch'] == id2][['y']].to_numpy()
    Y = df[df['Batch'] == id1][['y']].to_numpy()

    np.random.seed(0)
    combined = np.concatenate([X, Y])
    n = len(X)
    obs_diff = np.max(X) - np.max(Y)
    
    perm_diffs = []
    for _ in range(n_permutations):
        np.random.shuffle(combined)
        X_permuted = combined[:n]
        Y_permuted = combined[n:]
        max_diff = np.max(X_permuted) - np.max(Y_permuted)
        perm_diffs.append(max_diff)
        
    p_value = np.mean(perm_diffs > obs_diff)
    return np.array(perm_diffs), obs_diff, p_value

def run_permtest_experiment(n_runs=10):
    N_INITIAL = 50
    EPOCHS = 10
    BATCH_SIZE = 10
    DIM = 2
    LOWER = 0
    UPPER = np.pi

    p_value_matrix = np.zeros((n_runs, 10))

    for run_idx in range(n_runs):
        optimizer = BayesianOptimization(fun=Michalewicz(dim=DIM, negate=True), 
                                         batch_size=BATCH_SIZE, 
                                         dim=DIM, 
                                         epochs=EPOCHS, 
                                         n_init=N_INITIAL, 
                                         lower_bound=LOWER,
                                         upper_bound=UPPER,
                                         seed=np.random.randint(0, 10000),  # Change seed for each run
                                         acqf_type='qUCB')

        x_max, y_max = optimizer.run()
        data = optimizer.get_data()
        full = optimizer.format(data, dim=DIM, n_init=N_INITIAL, batch_size=BATCH_SIZE, epochs=EPOCHS)

        for id2 in range(1, 11):
            _, _, p_value = permutation_test_max(id2=id2, id1=0, n_permutations=10000, df=full)
            p_value_matrix[run_idx, id2 - 1] = p_value  

    mean_p_values = np.mean(p_value_matrix, axis=0)
    std_p_values = np.std(p_value_matrix, axis=0)

    return mean_p_values, std_p_values, p_value_matrix

mean_p_values, std_p_values, p_value_matrix = run_permtest_experiment(n_runs=10)
print(f"Mean p-values for each id2: {mean_p_values}")
print(f"Standard deviation of p-values for each id2: {std_p_values}")

Mean p-values for each id2: [0.01621 0.02036 0.0508  0.02506 0.02507 0.0223  0.02459 0.02475 0.02039
 0.02449]
Standard deviation of p-values for each id2: [0.01071937 0.00838024 0.08551333 0.00106508 0.00150735 0.00666108
 0.00099644 0.00088685 0.00868498 0.00071617]


In [7]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

def run_ttest_experiment(n_runs=10):

    N_INITIAL = 50
    EPOCHS = 10
    BATCH_SIZE = 10
    DIM = 2
    LOWER = 0
    UPPER = np.pi
    ttest_p_value_matrix = np.zeros((n_runs, 10))

    for run_idx in range(n_runs):
        optimizer = BayesianOptimization(fun=Michalewicz(dim=DIM, negate=True), 
                                         batch_size=BATCH_SIZE, 
                                         dim=DIM, 
                                         epochs=EPOCHS, 
                                         n_init=N_INITIAL, 
                                         lower_bound=LOWER,
                                         upper_bound=UPPER,
                                         seed=np.random.randint(0, 10000), # random seed
                                         acqf_type='qUCB')


        x_max, y_max = optimizer.run()
        data = optimizer.get_data()
        full = optimizer.format(data, dim=DIM, n_init=N_INITIAL, batch_size=BATCH_SIZE, epochs=EPOCHS)

        batch_0 = full[full['Batch'] == 0]['y']

        for t in range(1, 11):
            batch_t = full[full['Batch'] == t]['y']
            t_stat, p_value = ttest_ind(batch_t, batch_0, alternative='greater')
            ttest_p_value_matrix[run_idx, t - 1] = p_value 


    mean_p_values = np.mean(ttest_p_value_matrix, axis=0)
    std_p_values = np.std(ttest_p_value_matrix, axis=0)

    return mean_p_values, std_p_values, ttest_p_value_matrix


mean_p_values, std_p_values, ttest_p_value_matrix = run_ttest_experiment(n_runs=10)
print(f"Mean p-values for each batch comparison: {mean_p_values}")
print(f"Standard deviation of p-values for each batch comparison: {std_p_values}")

Mean p-values for each batch comparison: [0.16216649 0.22993831 0.16181838 0.09200665 0.1086186  0.11046637
 0.10024114 0.20029243 0.18246629 0.17061654]
Standard deviation of p-values for each batch comparison: [0.19446032 0.20215681 0.13066315 0.04937246 0.08952027 0.09509257
 0.06794828 0.14076973 0.1266789  0.14252935]
